In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')
df_48ind.replace(-99.99, np.nan, inplace = True)
df_48ind.replace(-999, np.nan, inplace = True)
df_48ind.dropna(inplace = True)

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap = market_cap.loc[df_48ind.index]


df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)


df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)


# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)



In [10]:
market_weights.multiply(df_48ind.shift(-1), axis=1)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-0.000373,0.160492,0.071758,0.040215,0.027083,0.075340,0.001730,0.068113,0.393579,0.047907,...,0.120909,0.152741,0.014726,0.613648,0.005588,0.124005,0.107483,0.017385,0.168818,0.000015
1970-07-01,-0.000191,0.011293,-0.021050,-0.016314,0.005470,0.072663,0.011434,0.035504,0.357265,0.044728,...,0.141682,0.140190,0.026808,0.272840,0.042290,0.135367,0.018788,0.011006,0.094603,0.001974
1970-08-01,0.008208,0.149928,0.048990,0.026780,0.004140,0.041591,0.063646,0.042079,0.526725,0.074351,...,0.003451,0.150312,0.064528,0.375880,0.089774,-0.008416,0.105913,0.029186,0.149695,0.000729
1970-09-01,-0.001929,-0.000574,0.049218,-0.005671,0.080146,-0.030922,-0.012255,-0.017049,-0.013142,-0.023426,...,-0.048844,-0.085557,-0.030433,-0.048144,-0.022598,-0.111637,-0.117296,-0.018622,-0.127955,-0.002682
1970-10-01,-0.001069,0.161161,0.075624,-0.004648,0.074774,0.047017,0.042250,0.022588,0.408343,0.016803,...,0.024248,0.065663,-0.004688,0.439233,0.026393,0.096286,0.073851,0.016037,0.127673,-0.001241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.005409,-0.056936,-0.003056,-0.034004,-0.020728,-0.013478,0.024723,-0.001271,-0.000766,0.024038,...,-0.005908,-0.115951,-0.010580,0.056256,-0.026945,-0.171762,0.124900,-0.014100,-0.171900,-0.048836
2023-10-01,-0.002257,0.049015,0.036773,0.030530,0.024925,0.005202,0.092487,0.004829,0.074739,0.060935,...,0.018237,0.225864,0.095638,0.522984,0.121599,0.790775,0.151608,0.014607,0.373388,0.187422
2023-11-01,0.006429,0.048244,0.017814,0.004245,0.004715,0.008215,0.035184,0.005415,0.004926,0.028443,...,0.006194,0.129330,0.056618,0.497599,0.081245,0.580813,-0.023857,0.023977,0.268973,0.041717


In [11]:
market_weights * df_48ind.shift(-1)

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-0.000373,0.160492,0.071758,0.040215,0.027083,0.075340,0.001730,0.068113,0.393579,0.047907,...,0.120909,0.152741,0.014726,0.613648,0.005588,0.124005,0.107483,0.017385,0.168818,0.000015
1970-07-01,-0.000191,0.011293,-0.021050,-0.016314,0.005470,0.072663,0.011434,0.035504,0.357265,0.044728,...,0.141682,0.140190,0.026808,0.272840,0.042290,0.135367,0.018788,0.011006,0.094603,0.001974
1970-08-01,0.008208,0.149928,0.048990,0.026780,0.004140,0.041591,0.063646,0.042079,0.526725,0.074351,...,0.003451,0.150312,0.064528,0.375880,0.089774,-0.008416,0.105913,0.029186,0.149695,0.000729
1970-09-01,-0.001929,-0.000574,0.049218,-0.005671,0.080146,-0.030922,-0.012255,-0.017049,-0.013142,-0.023426,...,-0.048844,-0.085557,-0.030433,-0.048144,-0.022598,-0.111637,-0.117296,-0.018622,-0.127955,-0.002682
1970-10-01,-0.001069,0.161161,0.075624,-0.004648,0.074774,0.047017,0.042250,0.022588,0.408343,0.016803,...,0.024248,0.065663,-0.004688,0.439233,0.026393,0.096286,0.073851,0.016037,0.127673,-0.001241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.005409,-0.056936,-0.003056,-0.034004,-0.020728,-0.013478,0.024723,-0.001271,-0.000766,0.024038,...,-0.005908,-0.115951,-0.010580,0.056256,-0.026945,-0.171762,0.124900,-0.014100,-0.171900,-0.048836
2023-10-01,-0.002257,0.049015,0.036773,0.030530,0.024925,0.005202,0.092487,0.004829,0.074739,0.060935,...,0.018237,0.225864,0.095638,0.522984,0.121599,0.790775,0.151608,0.014607,0.373388,0.187422
2023-11-01,0.006429,0.048244,0.017814,0.004245,0.004715,0.008215,0.035184,0.005415,0.004926,0.028443,...,0.006194,0.129330,0.056618,0.497599,0.081245,0.580813,-0.023857,0.023977,0.268973,0.041717


In [35]:
df_48ind.shift(-1).dropna()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-0.81,5.57,5.84,6.76,3.50,12.52,0.53,13.27,4.60,8.28,...,6.66,8.39,4.32,11.78,1.00,6.63,10.58,9.67,10.37,0.12
1970-07-01,-0.31,0.37,-1.71,-2.71,0.65,11.41,3.23,6.50,4.22,7.25,...,7.75,8.44,6.61,5.11,7.03,6.39,1.59,5.65,5.65,14.45
1970-08-01,14.45,5.00,4.03,4.47,0.51,6.22,19.18,7.29,6.38,11.96,...,0.19,8.96,16.36,6.81,16.07,-0.40,8.72,14.86,8.70,5.75
1970-09-01,-3.56,-0.02,4.31,-1.02,10.35,-4.35,-3.74,-2.91,-0.16,-3.68,...,-2.62,-4.95,-7.58,-0.87,-3.95,-5.22,-9.97,-9.39,-7.39,-19.30
1970-10-01,-1.80,5.60,6.68,-0.84,10.05,6.52,11.30,3.76,4.89,2.47,...,1.36,3.65,-1.05,7.77,4.16,4.75,6.04,7.36,7.11,-8.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.84,-4.28,-6.43,-5.13,-2.43,-7.38,-11.17,-6.19,-6.81,-4.41,...,-2.57,-5.15,-2.69,-6.16,-4.45,-3.92,2.57,-10.45,-3.49,-2.72
2023-09-01,-5.83,-4.98,-0.39,-4.34,-4.00,-14.31,2.77,-2.05,-0.05,4.21,...,-3.46,-6.31,-0.90,0.74,-1.71,-2.89,3.49,-9.05,-6.31,-1.77
2023-10-01,-2.29,4.26,4.79,3.90,4.76,5.66,11.08,7.90,4.97,10.63,...,10.43,12.31,7.94,6.96,7.67,13.16,3.93,9.95,13.51,6.64


In [39]:
df4 = df_48ind.shift(-1).dropna()
df4

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-0.81,5.57,5.84,6.76,3.50,12.52,0.53,13.27,4.60,8.28,...,6.66,8.39,4.32,11.78,1.00,6.63,10.58,9.67,10.37,0.12
1970-07-01,-0.31,0.37,-1.71,-2.71,0.65,11.41,3.23,6.50,4.22,7.25,...,7.75,8.44,6.61,5.11,7.03,6.39,1.59,5.65,5.65,14.45
1970-08-01,14.45,5.00,4.03,4.47,0.51,6.22,19.18,7.29,6.38,11.96,...,0.19,8.96,16.36,6.81,16.07,-0.40,8.72,14.86,8.70,5.75
1970-09-01,-3.56,-0.02,4.31,-1.02,10.35,-4.35,-3.74,-2.91,-0.16,-3.68,...,-2.62,-4.95,-7.58,-0.87,-3.95,-5.22,-9.97,-9.39,-7.39,-19.30
1970-10-01,-1.80,5.60,6.68,-0.84,10.05,6.52,11.30,3.76,4.89,2.47,...,1.36,3.65,-1.05,7.77,4.16,4.75,6.04,7.36,7.11,-8.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.84,-4.28,-6.43,-5.13,-2.43,-7.38,-11.17,-6.19,-6.81,-4.41,...,-2.57,-5.15,-2.69,-6.16,-4.45,-3.92,2.57,-10.45,-3.49,-2.72
2023-09-01,-5.83,-4.98,-0.39,-4.34,-4.00,-14.31,2.77,-2.05,-0.05,4.21,...,-3.46,-6.31,-0.90,0.74,-1.71,-2.89,3.49,-9.05,-6.31,-1.77
2023-10-01,-2.29,4.26,4.79,3.90,4.76,5.66,11.08,7.90,4.97,10.63,...,10.43,12.31,7.94,6.96,7.67,13.16,3.93,9.95,13.51,6.64


In [40]:
standardized_mom

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1970-06-01,-1.552789,-1.706255,-0.666595,-0.834907,0.006561,-2.214555,-1.607764,-2.820273,-2.083862,-2.493151,...,-2.132580,-3.384300,-3.375765,-2.010064,-3.060148,-0.807418,-1.863005,-2.101533,-2.175295,-4.111958
1970-07-01,-1.181630,-0.914575,-0.225191,-0.334593,0.392706,-1.473238,-1.239007,-1.806717,-1.581113,-1.671006,...,-1.597521,-2.312125,-2.629238,-0.963896,-2.540701,-0.189034,-0.774564,-1.355132,-1.334394,-3.497985
1970-08-01,-1.406006,-1.177623,-0.562015,-0.862012,0.236186,-1.291817,-1.526623,-1.943718,-1.615548,-1.473512,...,-1.506435,-2.165850,-2.617322,-1.126296,-2.633374,-0.486775,-1.152582,-1.668547,-1.474689,-3.245490
1970-09-01,-0.770439,-0.823934,-0.620900,-0.558210,0.067013,-1.143117,-0.884062,-1.614750,-1.183548,-0.878993,...,-1.563159,-1.625913,-1.780652,-0.730250,-1.805491,-0.368052,-1.066597,-1.047378,-0.987988,-2.728855
1970-10-01,-1.784084,-1.330381,-0.740084,-1.273106,-0.029992,-1.529591,-1.481737,-2.051986,-1.594887,-1.302486,...,-1.896957,-2.130949,-2.756163,-1.103245,-2.422364,-1.112637,-1.888695,-1.511340,-1.754411,-3.685588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.861869,-0.699700,-0.169132,-0.360004,0.010778,-0.018598,0.655186,0.825862,0.113706,0.371124,...,0.218189,0.505092,0.714851,0.061318,0.310318,-0.055355,-0.121814,0.294214,0.158182,0.798994
2023-10-01,-1.829058,-1.729074,-0.522915,-1.139839,-0.742299,-0.682475,0.232911,0.374884,-0.243790,0.099518,...,-0.279781,-0.168803,-0.039447,0.002121,-0.273288,-0.815832,-0.610983,-0.238027,-0.613004,0.276951
2023-11-01,-2.061342,-1.646673,-0.660000,-1.102570,-0.781195,-0.661024,0.364117,0.051329,-0.522399,-0.180638,...,-0.284690,-0.058454,0.109755,0.212716,-0.193446,-0.442371,-0.478336,-0.268669,-0.400397,0.260728


In [32]:
standardized_market_caps = standardized_market_caps.loc[standardized_mom.index]
standardized_BtoM = standardized_BtoM.loc[standardized_mom.index]
standardized_mom = standardized_mom.loc[standardized_mom.index]
market_weights = market_weights.loc[standardized_mom.index]
df_48ind = df_48ind.loc[standardized_mom.index]

def maximize_utility(MC, BM, MOM, r, T, N, gamma):
    
    def u(x, gamma):
        return x ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM) * (r),axis=1), gamma))


    # Initialisation des coefficients beta
    initial_beta = np.array([0.3, 0.3, 0.3])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x

    return optimal_beta

maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind, df_48ind.shape[0], df_48ind.shape[1], 5)

array([-12622.59734427, -44863.59197825,  -4351.55659974])

In [41]:
standardized_market_caps = standardized_market_caps.loc[df4.index]
standardized_BtoM = standardized_BtoM.loc[df4.index]
standardized_mom = standardized_mom.loc[df4.index]
market_weights = market_weights.loc[df4.index]
df_48ind = df_48ind.loc[df4.index]

def maximize_utility(MC, BM, MOM, r, T, N, gamma):
    
    def u(x, gamma):
        return x ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights
        return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM) * (r),axis=1), gamma))


    # Initialisation des coefficients beta
    initial_beta = np.array([0.3, 0.3, 0.3])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x

    return optimal_beta

maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, df_48ind, df_48ind.shape[0], df_48ind.shape[1], 5)

array([-12642.22859227, -44933.36474818,  -4358.32465327])

In [36]:
#-(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)) * (r), gamma))

MC = standardized_market_caps
BM = standardized_BtoM
MOM = standardized_mom
r = df_48ind.shift(-1).dropna()
T = df_48ind.shape[0]
N = df_48ind.shape[1]
gamma = 5

wi_t = market_weights
beta = [0.5, 0.5, 0.5]

def u(x, gamma):
    return x ** (1 - gamma) / (1 - gamma)


-(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM) * (r),axis=1), gamma))

inf